# Data preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
key = "SK_ID_CURR"

## Presious application

In [ ]:
prev_app = pd.read_csv("data/previous_application.csv")

In [ ]:
prev_app.head()

In [ ]:
prev_app["NAME_CONTRACT_STATUS"].value_counts()

In [ ]:
# AMT_ACCEPT_RATE: credit get compared to the amount the client applied
prev_app['AMT_ACCEPT_RATE'] = prev_app['AMT_CREDIT']/prev_app['AMT_APPLICATION']

In [ ]:
prev_app_agg = {
    "NAME_CONTRACT_STATUS": ["count"],
    "AMT_ANNUITY": ["sum", "mean"],
    "AMT_APPLICATION": ["sum", "mean"]    
}

prev_app_status = prev_app[["NAME_CONTRACT_STATUS", "SK_ID_CURR", "SK_ID_PREV", "AMT_ANNUITY", "AMT_APPLICATION"]].groupby(["SK_ID_CURR","NAME_CONTRACT_STATUS"]).agg(prev_app_agg).reset_index()

prev_app_status.columns = [
    f"{col[0]}_{col[1].upper()}" if col[1] else col[0].upper()
    for col in prev_app_status.columns
]


pre_app_agg = prev_app_status.pivot_table(
        index="SK_ID_CURR",
        columns="NAME_CONTRACT_STATUS",
        values=["NAME_CONTRACT_STATUS_COUNT", "AMT_ANNUITY_SUM", "AMT_ANNUITY_MEAN", "AMT_APPLICATION_SUM", "AMT_APPLICATION_MEAN"],
        aggfunc="sum",     # combine if there are duplicates
        fill_value=0       # fill missing with 0
    ).reset_index()

pre_app_agg.columns = [
    f"{col[0]}_{col[1].upper()}" if col[1] else col[0].upper()
    for col in pre_app_agg.columns
]


In [ ]:
pre_app_agg.to_csv("etl/previous_applications_agg.csv", header=True, index=False)